In [1]:
from river import stream, compose, preprocessing, evaluate, metrics
from river.tree import HoeffdingTreeClassifier
import datetime

dataset_path = "../dataset/AEP_hourly.csv"

def float_converter(a):
    try:
        a=float(a)
    except ValueError:
        a=None
    return a

def divide_date(x):#feature enginnering
    try:
        dt = datetime.datetime.strptime(x['Datetime'], "%Y-%m-%d %H:%M:%S")
        x["Year"]=dt.year
        x["Month"]=dt.month
        x["Day"]=dt.day
        x["Hour"]=dt.hour
    except:
        x["Year"]=x["Month"]=x["Day"]=x["Hour"]=None
    return x

energy_consumption = stream.iter_csv(dataset_path, target="AEP_MW", drop_nones=True, converters={'AEP_MW':float_converter})

model= (
    compose.FuncTransformer(divide_date) + 
    compose.Select('Month','Day','Hour') | 
    preprocessing.OneHotEncoder(drop_zeros=False) | 
    compose.Discard("Datetime")
)

model|= HoeffdingTreeClassifier(grace_period=50)

print(evaluate.progressive_val_score(dataset=energy_consumption, model=model, metric=metrics.Accuracy()))
                            

Accuracy: 0.00%
